In [1]:
%%capture
!pip install crewai
!pip install crewai-tools
!pip install langchain-community
!pip install langchain-huggingface
!pip install sentence-transformers

In [3]:
%%capture
from crewai import Agent, Task, Crew, Process
from crewai import LLM
from crewai_tools import PDFSearchTool, SerperDevTool
from google.colab import userdata

In [15]:
api_key = userdata.get('OPENAI_API_KEY')
llm = LLM(model="gpt-4o", api_key=api_key)

### The Role of Tools: Agent-Centric versus Task-Centric

A key decision in CrewAI is *how* to give tools to your agents. There are two main strategies, and understanding the difference is crucial for building robust applications.

**1. Agent-Centric Approach (The Generalist):**
This is the most common method. You give an agent a 'toolbox' containing all the tools it might possibly need. The agent then uses its intelligence (the LLM's reasoning ability) to decide which tool is best for the situation at hand. This is flexible but can sometimes lead to the agent making mistakes or using tools inefficiently.

![Diagram 1](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/NPYpo2eWUERulpzrm9xYtw/Adobe%20Express%20-%20file.jpg)

**2. Task-Centric Approach (The Specialist):**
In this more advanced approach, you don't give the tools to the agent directly. Instead, you attach specific tools to the specific **Tasks** that require them. When the agent starts a task, it is temporarily granted access to only the tools needed for that job. This creates a much more focused and predictable workflow.

![Diagram 2.jpg](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/olP9UG8s4B7QnOJhCRH4aA/Diagram%202.jpg)


In [16]:
import os
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')

In [17]:
web_search_tool = SerperDevTool()

In [18]:
import warnings
warnings.filterwarnings('ignore') #Keeps Jupyter Notebook clean (not part of functionality)

pdf_search_tool = PDFSearchTool(
    pdf="/content/The_Daily_Dish_FAQ.pdf",
    config=dict(
        embedder=dict(
            provider="huggingface",
            config=dict(
                model="sentence-transformers/all-MiniLM-L6-v2"
            )
        )
    )
)

### Approach 1: The Standard Method (Agent-Centric Tools)

First, we'll build the chatbot using the conventional approach where we give our agent a toolbox with all the necessary tools. The agent will be responsible for deciding whether to search the PDF or generate a response with just it's LLM.

#### **Step 1.1: Create the Agent**

We define an `Inquiry Specialist Agent` whose job is to answer questions. Notice that the `tools` parameter is a list containing the `pdf_search_tool` and `web_search_tool`.


In [19]:
agent_centric_agent = Agent(
    role="The Daily Dish Inquiry Specialist",
    goal="""Accurately answer customer questions about The Daily Dish restaurant.
    You must decide whether to use the restaurant's FAQ PDF or a web search to find the best answer.""",
    backstory="""You are an AI assistant for 'The Daily Dish'.
    You have access to two tools: one for searching the restaurant's FAQ document and another for searching the web.
    Your job is to analyze the user's question and choose the most appropriate tool to find the information needed to provide a helpful response.""",
    tools=[pdf_search_tool, web_search_tool],
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [20]:
agent_centric_task = Task(
    description="Answer the following customer query: '{customer_query}'. "
                "Analyze the question and use the tools at your disposal (PDF search or web search) to find the most relevant information. "
                "Synthesize the findings into a clear and friendly response.",
    expected_output="A comprehensive and well-formatted answer to the customer's query.",
    agent=agent_centric_agent
)

In [21]:
agent_centric_crew = Crew(
    agents=[agent_centric_agent],
    tasks=[agent_centric_task],
    process=Process.sequential,
    verbose=False
)

In [22]:
print("\nWelcome to The Daily Dish Chatbot!")
print("What would you like to know? (Type 'exit' to quit)")

while True:
    user_input = input("\nYour question: ").lower()
    if user_input == 'exit':
        print("Thank you for chatting. Have a great day!")
        break

    if not user_input:
        print("Please type a question.")
        continue

    try:
        # Here we use our more advanced, task-centric crew
        result_agent_centric = agent_centric_crew.kickoff(inputs={'customer_query': user_input})
        print("\n--- The Daily Dish Assistant ---")
        print(result_agent_centric)
        print("--------------------------------")
    except Exception as e:
        print(f"An error occurred: {e}")


Welcome to The Daily Dish Chatbot!
What would you like to know? (Type 'exit' to quit)

Your question: what is the timing 


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Daily Dish Inquiry Specialist                                                                       │
│                                                                                                                 │
│  Task: Answer the following customer query: 'what is the timing '. Analyze the question and use the tools at    │
│  your disposal (PDF search or web search) to find the most relevant information. Synthesize the findings into   │
│  a clear and friendly response.                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Daily Dish Inquiry Specialist                                                                       │
│                                                                                                                 │
│  Thought: Thought: The customer is asking about the timings of The Daily Dish restaurant. This type of          │
│  information is likely to be available in the restaurant's FAQ document. I will search the FAQ PDF to find the  │
│  restaurant's operating hours.                                                                                  │
│                                                                                                                 │
│  Using Tool: Search a PDF's content                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"timing\"}"                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│  No relevant content found.                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Daily Dish Inquiry Specialist                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The Daily Dish is located at 123 Culinary Avenue, Foodie Town, FT 54321. If you need further assistance, feel  │
│  free to reach out at (555) 123-4567.                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- The Daily Dish Assistant ---
The Daily Dish is located at 123 Culinary Avenue, Foodie Town, FT 54321. If you need further assistance, feel free to reach out at (555) 123-4567.
--------------------------------

Your question: exit 


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Daily Dish Inquiry Specialist                                                                       │
│                                                                                                                 │
│  Task: Answer the following customer query: 'exit '. Analyze the question and use the tools at your disposal    │
│  (PDF search or web search) to find the most relevant information. Synthesize the findings into a clear and     │
│  friendly response.                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Daily Dish Inquiry Specialist                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The user's query "exit " seems to be incomplete or lacking specific context related to The Daily Dish. This    │
│  does not seem to be a question directly related to The Daily Dish restaurant. The query might be a result of   │
│  a misunderstanding or error.                                                                                   │
│                                                                                                                 │
│  Given the context and my capabilities, I'm unable to provide further assistance without more specific          │
│  information or a clarified query related to The Daily Dish.                                                    │
│                                                                                                                 │
│  If the query was meant to accomplish something specific, please provide more details or clarify your question  │
│  for a more accurate response.                                                                                  │
│                                                                                                                 │
│  For any restaurant-specific questions or services, you can visit the restaurant's website or contact them      │
│  directly for comprehensive assistance.                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- The Daily Dish Assistant ---
The user's query "exit " seems to be incomplete or lacking specific context related to The Daily Dish. This does not seem to be a question directly related to The Daily Dish restaurant. The query might be a result of a misunderstanding or error.

Given the context and my capabilities, I'm unable to provide further assistance without more specific information or a clarified query related to The Daily Dish. 

If the query was meant to accomplish something specific, please provide more details or clarify your question for a more accurate response. 

For any restaurant-specific questions or services, you can visit the restaurant's website or contact them directly for comprehensive assistance.
--------------------------------

Your question: exit
Thank you for chatting. Have a great day!


### Approach 2: A More Focused Method (Task-Centric Tools)

Now, we'll refactor our solution to use a task-centric approach. We will create a multi-step process where each step (Task) has its own dedicated tool. This makes the agent's job simpler and the overall workflow more reliable.

Our new workflow will have two tasks:
1.  **Search the FAQ:** This task will *only* use the `PDFSearchTool`.
2.  **Draft the Response:** This task will use the information from the first two tasks to write the final answer. It needs no tools.

#### **Step 2.1: Create the Agent**

This time, we create a `Customer Service Specialist` agent. Notice the critical difference: the `tools` list is **empty**. We are not giving the agent its toolbox upfront. However, even if we gave the agent many many tools, they would simply be completely overriden by the tools in the task.


In [23]:
task_centric_agent = Agent(
    role="Customer Service Specialist",
    goal="Provide exceptional customer service by following a multi-step process to answer customer questions accurately.",
    backstory="""You are an AI assistant for 'The Daily Dish'.
    You are an expert at following instructions. You will be given a sequence of tasks to complete.
    For each task, you will be provided with the specific tool needed to accomplish it.
    Your job is to execute each task diligently and pass the results to the next step.""",
    tools=[], # The agent is not given any tools directly
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [24]:
faq_search_task = Task(
    description="Search the restaurant's FAQ PDF for information related to the customer's query: '{customer_query}'.",
    expected_output="A snippet of the most relevant information from the PDF, or a statement that the information was not found.",
    tools=[pdf_search_tool], # Tool assigned directly to the task
    agent=task_centric_agent
)

response_drafting_task = Task(
    description="Using the information gathered from the FAQ search, draft a friendly and comprehensive response to the customer's query: '{customer_query}'.",
    expected_output="The final, customer-facing response.",
    agent=task_centric_agent,
    context=[faq_search_task]
)

In [25]:
task_centric_crew = Crew(
    agents=[task_centric_agent],
    tasks=[faq_search_task, response_drafting_task],
    process=Process.sequential,
    verbose=True
)

In [26]:
print("\nWelcome to The Daily Dish Chatbot!")
print("What would you like to know? (Type 'exit' to quit)")

while True:
    user_input = input("\nYour question: ").lower()
    if user_input == 'exit':
        print("Thank you for chatting. Have a great day!")
        break

    if not user_input:
        print("Please type a question.")
        continue

    try:
        # Here we use our more advanced, task-centric crew
        result_task_centric = task_centric_crew.kickoff(inputs={'customer_query': user_input})
        print("\n--- The Daily Dish Assistant ---")
        print(result_task_centric)
        print("--------------------------------")
    except Exception as e:
        print(f"An error occurred: {e}")


Welcome to The Daily Dish Chatbot!
What would you like to know? (Type 'exit' to quit)

Your question: what is the timing 


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b35f1915-bea1-4686-b822-c1cee0018bc3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Service Specialist                                                                             │
│                                                                                                                 │
│  Task: Search the restaurant's FAQ PDF for information related to the customer's query: 'what is the timing '.  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Service Specialist                                                                             │
│                                                                                                                 │
│  Thought: Thought: I need to search the PDF for information related to the timing of the restaurant.            │
│                                                                                                                 │
│  Using Tool: Search a PDF's content                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Service Specialist                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Thank you for reaching out to us with your question! The Daily Dish is located at 123 Culinary Avenue, Foodie  │
│  Town, FT 54321. If you have any more questions or need further assistance, please feel free to ask. We're      │
│  here to help!                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 770e2323-37cb-45fa-aa31-04fc0121268a                                                                     │
│  Agent: Customer Service Specialist                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b35f1915-bea1-4686-b822-c1cee0018bc3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Thank you for reaching out to us with your question! The Daily Dish is located at 123 Culinary   │
│  Avenue, Foodie Town, FT 54321. If you have any more questions or need further assistance, please feel free to  │
│  ask. We're here to help!                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- The Daily Dish Assistant ---
Thank you for reaching out to us with your question! The Daily Dish is located at 123 Culinary Avenue, Foodie Town, FT 54321. If you have any more questions or need further assistance, please feel free to ask. We're here to help!
--------------------------------

Your question: exit
Thank you for chatting. Have a great day!


<!-- ## Conclusion

In this lab, you built a sophisticated customer service chatbot and, more importantly, explored a fundamental concept in CrewAI: the strategic assignment of tools.

You saw two approaches:
1.  **Agent-Centric:** Flexible and easy to set up, but relies on the agent's reasoning to select tools, which can be inefficient or unpredictable in complex scenarios.
2.  **Task-Centric:** More structured and robust. By assigning tools directly to the tasks that need them, you create a clear, deterministic, and efficient workflow.

**Key Advantages of Task-Centric Tools:**
- **Focus & Efficiency:** The agent doesn't waste time or processing power deciding which tool to use. It's told exactly what to use for each step.
- **Clarity & Maintainability:** The workflow is explicit and easy to follow. Anyone reading the code can see precisely which task uses which tool.
- **Control & Security:** This pattern allows for fine-grained control, granting an agent access to a powerful tool only for the specific duration of a single task.

Although the agent-centric approach has its place, mastering the task-centric method is a key step toward building truly professional, production-grade multi-agent systems with CrewAI.
 -->

## Conclusion

In this lab, you built a sophisticated customer service chatbot and, more importantly, explored a fundamental concept in CrewAI: the strategic assignment of tools.

You saw two approaches:
1. **Agent‑Centric:** Flexible and easy to set up, but relies on the agent's reasoning to select tools, which can be inefficient or unpredictable in complex scenarios.  
2. **Task‑Centric:** More structured and robust. By assigning tools directly to the tasks that need them, you create a clear, deterministic, and efficient workflow.

| Aspect             | Agent‑Centric Output                                                   | Task‑Centric Output                                                       |
|--------------------|-------------------------------------------------------------------------|----------------------------------------------------------------------------|
| **Predictability** | Varies run‑to‑run based on the LLM’s tool choice and phrasing.          | Consistent across runs thanks to fixed task‑to‑tool mapping.              |
| **Debuggability**  | Harder to trace—calls and errors are buried in the LLM’s reasoning.     | Easy to debug—each task’s inputs, outputs, and errors are explicitly logged. |
| **Reusability**    | You get a free‑form text blob that you must parse yourself.             | You get structured intermediate results (for example, JSON or code blocks) ready for reuse. |
| **Structure**¹     | Search and formatting are blended in one step.                          | A separate formatting task produces a clean, structured final message.    |

This “Structure” difference exists only because we introduced a dedicated formatting task in the Task‑Centric workflow.

**Key Advantages of Task‑Centric Tools:**
- **Focus & Efficiency:** The agent doesn't waste time or processing power deciding which tool to use. It's told exactly what to use for each step.  
- **Clarity & Maintainability:** The workflow is explicit and easy to follow. Anyone reading the code can see precisely which task uses which tool.  
- **Control & Security:** This pattern allows for fine‑grained control, granting an agent access to a powerful tool only for the specific duration of a single task.

While the agent‑centric approach has its place, mastering the task‑centric method is a key step toward building truly professional, production‑grade multi‑agent systems with CrewAI.
